In [18]:
import os
import numpy as np
import h5py
import lal
from lal import LIGOTimeGPS
from lal import series as lalseries

from datetime import datetime, timezone
from ligo.lw import ligolw, lsctables, utils
from ligo.lw.param import Param

# Output of ourself
## parameters
mass1, mass2, spin1, spin2,
## snr timeseries
timeseries, amplitude, phase, trigger time
## PSD 
PSD

# hdf file format
- event
  - Parameters
    - Mass1
    - Mass2
    - Spin1x
    - Spin1y
    - Spin1z
    - Spin2x
    - Spin2y
    - Spin2z
  - SNR_ts
    - {ifo}-SNR_ts
    - {ifo}-merger_time
    - {ifo}-start_time
    - sampling_rate
  - PSD
    - {ifo}-PSD
    - {ifo}-sampling_frequency
    - {ifo}-t0
    - f0




# write XML file

In [19]:
# Initialize the file
def new_row(table):
    """ Construct a table all of the value in column is None. """
    row = table.RowType()
    for col in table.validcolumns:
        setattr(row, col, None)
    return row

In [20]:
def make_snr_timeseries_from_start(series,ifo, start_time, dt):
    """
    series     : 1D complex array (SNR time series)
    start_time : 該 series 第 0 個 sample 的 GPS time (float)
    dt         : time step (= 1/sampling_rate)

    這裡的 epoch = start_time，
    所以第 i 個 sample 的時間 = start_time + i * dt。
    """
    series = np.asarray(series, dtype=np.complex64)

    epoch = LIGOTimeGPS(float(start_time))

    ts = lal.CreateCOMPLEX8TimeSeries(
        name=f"snr",
        epoch=epoch,
        f0=0.0,
        deltaT=dt,
        sampleUnits=lal.DimensionlessUnit,
        length=series.size,
    )
    ts.data.data[:] = series
    return ts

In [21]:
def make_psd_frequencyseries_from_start(psd, ifo, start_time, f0, df):
    """
    psd         : 1D real array (PSD)
    start_time  : start time to calculate PSD
    df          : frequency resolution (= 1/samping_frequency)
    """

    psd = np.asarray(psd, dtype=float)
    epoch = LIGOTimeGPS(float(start_time))
    lal_psd = lal.CreateREAL8FrequencySeries(
            name=f"{ifo}_PSD",
            epoch=epoch,
            f0=f0,
            deltaF=df,
            sampleUnits=lal.HertzUnit,
            length=len(psd),
        )
    lal_psd.data.data[:] = psd
    
    return lal_psd


In [22]:
''' 
Write coinc xml file, which contains 
sngl_inspiral:table 
coinc_inspiral:table 
coinc_event:table 
coinc_definer:table
coinc_event_map:table
time_slide:table
process:table
process_params:table
snr array
psd array
'''

def write_coinc_xml_from_snr(
    params_and_snr_ts_hdf_path,
    event_id,
    outfile,
    ifos=("H1", "L1", "K1"),
):
    """
    params_and_snr_ts_hdf_path: hdf file contains waveform parameter and snr timeseries
    - event
        - Parameters
            - Mass1
            - Mass2
            - Spin1x
            - Spin1y
            - Spin1z
            - Spin2x
            - Spin2y
            - Spin2z
        - SNR_ts
            - {ifo}-SNR_ts: complex snr timeseries
            - {ifo}-trigger_time: detector trigger time
            - {ifo}-start_time: snr timeseries start time
            - {ifo}-sampling_rate: sampling rate
        - PSD
            - {ifo}-PSD: PSD
            - {ifo}-start_time: start time to calculate PSD
            - {ifo}-delta_f: frecuency resolution
            - {ifo}-f0: low frequency cut-off 
            - f_final: high frequency cut-off 
    outfile: outptut xml file
    """

    # ---- 0. read waveform parameters ,snr timeseries and PSD file ----
    with h5py.File(params_and_snr_ts_hdf_path, "r") as hf:
        event = hf["event"]

        params_data = event["Parameters"]
        mass1 = params_data["Mass1"][()]
        mass2 = params_data["Mass2"][()]
        mchirp = (mass1*mass2)**0.6/(mass1+mass2)**0.2
        spin1x = params_data["Spin1x"][()]
        spin1y = params_data["Spin1y"][()]
        spin1z = params_data["Spin1z"][()]
        spin2x = params_data["Spin2x"][()]
        spin2y = params_data["Spin2y"][()]
        spin2z = params_data["Spin2z"][()]

        snr_data = event["SNR_ts"]
        snr_ts = {}
        start_time = {}
        trigger_time = {}
        amp = {}
        phase = {}
        dt = {}
        for ifo in ifos:
            snr_channel = f"{ifo}-SNR_ts"
            mt_channel = f"{ifo}-trigger_time"
            st_channel = f"{ifo}-start_time"
            sampling_rate = float(snr_data[f"{ifo}-sampling_rate"][()])

            snr_ts[ifo] = snr_data[snr_channel][()]          # complex array
            trigger_time[ifo] = float(snr_data[mt_channel][()])
            start_time[ifo] = float(snr_data[st_channel][()])

            peak_ind = np.abs(snr_ts[ifo]).argmax()
            amp[ifo] = float(np.abs(snr_ts[ifo][peak_ind]))
            phase[ifo] = float(np.angle(snr_ts[ifo][peak_ind]))
            dt[ifo] = 1/sampling_rate
            
        PSD_data = event["PSD"]
        psds = {}
        psds_start_time = {}
        psds_delta_f = {}
        f0 = {}
        # f_final = PSD_data["f_final"][()]
        for ifo in ifos:
            psd_channel = f"{ifo}-PSD"
            psd_start_channel = f"{ifo}-start_time"
            psd_delta_f_channel = f"{ifo}-delta_f"
            f0_channel = f"{ifo}-f0"
            psds[ifo] = PSD_data[psd_channel][()]
            psds_start_time[ifo] = PSD_data[psd_start_channel][()]
            psds_delta_f[ifo] = PSD_data[psd_delta_f_channel][()]
            f0[ifo] = PSD_data[f0_channel][()]

    # ---- 1. 建 LIGO_LW 文件與所有 tables ----
    xmldoc = ligolw.Document()
    xmlroot = ligolw.LIGO_LW()
    xmldoc.appendChild(xmlroot)

    process_table = lsctables.New(lsctables.ProcessTable)
    process_params_table = lsctables.New(lsctables.ProcessParamsTable)
    coinc_def_table = lsctables.New(lsctables.CoincDefTable)
    sngl_inspiral_table = lsctables.New(lsctables.SnglInspiralTable)
    coinc_table = lsctables.New(lsctables.CoincTable)
    coinc_inspiral_table = lsctables.New(lsctables.CoincInspiralTable)
    coinc_map_table = lsctables.New(lsctables.CoincMapTable)
    time_slide_table = lsctables.New(lsctables.TimeSlideTable)

    xmlroot.appendChild(process_table)
    xmlroot.appendChild(process_params_table)
    xmlroot.appendChild(coinc_def_table)
    xmlroot.appendChild(sngl_inspiral_table)
    xmlroot.appendChild(coinc_table)
    xmlroot.appendChild(coinc_inspiral_table)
    xmlroot.appendChild(coinc_map_table)
    xmlroot.appendChild(time_slide_table)

    # ---- 2. Process ----
    proc = new_row(process_table)
    proc.comment = ""

    proc.process_id = process_table.get_next_id()
    proc.program = "gstlal_inspiral"
    proc.version = "1.0"
    proc.cvs_repository = "local"
    proc.cvs_entry_time = None
    proc.comment = "Build coinc file from custom SNR time series"
    proc.username = os.getenv("USER", "unknown")
    now = datetime.now(timezone.utc)
    proc.start_time = lal.LIGOTimeGPS(now.timestamp())
    proc.end_time = None
    proc.is_online = 0
    for extra in ("ifos", "domain", "node", "uid", "gid", "pid", "unix_procid", "jobid"):
        if extra in process_table.validcolumns and getattr(proc, extra) is None:
            if extra in ("uid", "gid", "pid", "unix_procid", "jobid"):
                setattr(proc, extra, 0)
            elif extra == "ifos":
                setattr(proc, extra, ",".join(ifos))
            else:
                setattr(proc, extra, "")
    process_table.append(proc)

    # ---- 3. ProcessParams + SearchSummary ----
    ppr = new_row(process_params_table)
    ppr.process_id = proc.process_id
    if "program" in process_params_table.validcolumns:
        ppr.program = proc.program
    ppr.param = "description"
    ppr.type = "lstring"
    ppr.value = "MDC coinc XML for BAYESTAR from custom SNR"
    process_params_table.append(ppr)

    # ---- 4. CoincDef ----
    insp_def = new_row(coinc_def_table)
    insp_def.coinc_def_id = coinc_def_table.get_next_id()
    insp_def.search = "inspiral"
    insp_def.description = "sngl_inspiral<->coinc_event coincidences"
    insp_def.search_coinc_type = 0
    coinc_def_table.append(insp_def)

    # ---- 5. TimeSlide ----
    offsets = {ifo: 0.0 for ifo in ifos}
    time_slide_id = time_slide_table.append_offsetvector(offsets, proc)

    # ---- 6. CoincTable（network 事件）----
    coinc_row = new_row(coinc_table)
    coinc_row.coinc_event_id = event_id
    coinc_row.process_id = proc.process_id
    coinc_row.coinc_def_id = insp_def.coinc_def_id
    coinc_row.time_slide_id = time_slide_id
    inst_str = ",".join(ifos)
    coinc_row.instruments = inst_str

    per_ifo_peak = {ifo: float(np.abs(snr_ts[ifo]).max()) for ifo in ifos}
    net_snr = np.sqrt(sum(v * v for v in per_ifo_peak.values()))

    coinc_row.nevents = len(ifos)
    coinc_row.likelihood = None
    coinc_table.append(coinc_row)

    # ---- 7. CoincInspiral（network mass + net SNR）----
    c_insp_row = new_row(coinc_inspiral_table)
    c_insp_row.coinc_event_id = coinc_row.coinc_event_id
    c_insp_row.instruments = list(ifos)

    max_snr_ifo = max(amp, key=amp.get)
    mt = LIGOTimeGPS(trigger_time[max_snr_ifo])
    c_insp_row.end = mt
    c_insp_row.end_time = mt.gpsSeconds
    c_insp_row.end_time_ns = mt.gpsNanoSeconds

    c_insp_row.mass = mass1 + mass2
    c_insp_row.mchirp = mchirp
    c_insp_row.combined_far = 0.0
    c_insp_row.false_alarm_rate = 0.0
    c_insp_row.minimum_duration = None
    c_insp_row.snr = net_snr
    coinc_inspiral_table.append(c_insp_row)

    # ---- 8. sngl_inspiral + SNR(t) + CoincMap ----
    for ifo in ifos:
        series = snr_ts[ifo]
        abs_snr = amp[ifo]
        arg_snr = phase[ifo]

        srow = new_row(sngl_inspiral_table)
        srow.process_id = proc.process_id
        srow.ifo = ifo
        srow.mass1 = mass1
        srow.mass2 = mass2
        srow.spin1x = spin1x
        srow.spin1y = spin1y
        srow.spin1z = spin1z
        srow.spin2x = spin2x
        srow.spin2y = spin2y
        srow.spin2z = spin2z

        # 直接用你存的 peak time 當 trigger time
        mt = LIGOTimeGPS(trigger_time[ifo])
        srow.end_time = mt.gpsSeconds
        srow.end_time_ns = mt.gpsNanoSeconds

        srow.snr = abs_snr
        srow.coa_phase = arg_snr
        srow.f_final = 1024.0
        srow.eff_distance = 1.0

        # event_id
        srow.event_id = sngl_inspiral_table.get_next_id()
        sngl_inspiral_table.append(srow)

        # SNR series：epoch = start_time[ifo]
        tseries = make_snr_timeseries_from_start(series,ifo, start_time[ifo], dt[ifo])
        elem = lalseries.build_COMPLEX8TimeSeries(tseries)
        elem.appendChild(Param.from_pyvalue("event_id", srow.event_id))
        elem.appendChild(Param.from_pyvalue("instrument", ifo))
        xmlroot.appendChild(elem)

        # PSD series: epoch = PSD_start_time[ifo]
        df = psds_delta_f[ifo]
        psd_series = make_psd_frequencyseries_from_start(psds[ifo], ifo, psds_start_time[ifo], f0[ifo], df)
        elem = lalseries.build_REAL8FrequencySeries(psd_series)
        elem.appendChild(Param.from_pyvalue("instrument", ifo))
        xmlroot.appendChild(elem)

        # CoincMap
        map_row = new_row(coinc_map_table)
        map_row.coinc_event_id = coinc_row.coinc_event_id
        map_row.table_name = sngl_inspiral_table.tableName
        map_row.event_id = srow.event_id
        coinc_map_table.append(map_row)

    # ---- 9. 寫出 XML 檔 ----
    utils.write_filename(xmldoc, outfile)
    print(f"[coinc] Wrote BAYESTAR coinc XML to: {outfile}")

    return {
        "coinc_row": coinc_row,
        "coinc_inspiral_row": c_insp_row,
        "per_ifo_peak_snr": per_ifo_peak,
        "net_snr": net_snr,
    }


In [23]:
# input_file_name = "/home/chihyi.chang/Bayestar/large_real_event/GW250114/coinc_gw250114.hdf"
# output_file_name = "/home/chihyi.chang/Bayestar/large_real_event/GW250114/coinc_gw250114.xml"
event_label = "gw250114_p8k_q3"
input_file_name = f"/home/chiajui.chou/bayestar-test/coinc_{event_label}.hdf5"
output_file_name = f"/home/chiajui.chou/bayestar-test/coinc_{event_label}.xml"
write_coinc_xml_from_snr(
    params_and_snr_ts_hdf_path=input_file_name,
    event_id=0,
    outfile=output_file_name,
    ifos=("L1", "H1"),
)

[coinc] Wrote BAYESTAR coinc XML to: /home/chiajui.chou/bayestar-test/coinc_gw250114_p8k_q3.xml


{'coinc_row': <ligo.lw.lsctables.Coinc at 0x7f7e84808510>,
 'coinc_inspiral_row': <ligo.lw.lsctables.CoincInspiral at 0x7f7e848144c0>,
 'per_ifo_peak_snr': {'L1': 61.536134606697836, 'H1': 55.972167436085854},
 'net_snr': 83.184009219482}